In [8]:
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [9]:
with open('data.json', 'r', encoding='utf-8') as file:
    json_data = file.read()

data = json.loads(json_data)
devices_data = data["data"]

In [10]:
# 提取 devName、startTime 和 endTime
table_data = []

for device in devices_data:
    dev_name = device["devName"]
    resv_info = device["resvInfo"]
    for resv in resv_info:
        start_time = datetime.fromtimestamp(resv["startTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.fromtimestamp(resv["endTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        table_data.append([dev_name, start_time, end_time])

In [11]:
# 使用 pandas 创建 DataFrame
columns = ["devName", "startTime", "endTime"]
df = pd.DataFrame(table_data, columns=columns)

In [12]:
df["startTime"] = pd.to_datetime(df["startTime"])
df["endTime"] = pd.to_datetime(df["endTime"])
df['duration'] = (df['endTime'] - df['startTime']).dt.seconds / 3600

In [13]:
# 根据设备名称分组
groups = df.groupby("devName")

dev_names = df['devName'].unique()
name_to_index = {name: i for i, name in enumerate(dev_names)}

# 创建散点图
fig = go.Figure()

for name, group in groups:
    y_pos = name_to_index[name]
    for start_time, end_time in zip(group['startTime'], group['endTime']):
        fig.add_trace(
            go.Scatter(
                x=[start_time, end_time],
                y=[y_pos, y_pos],
                mode='lines',
                line=dict(color='orange', width=10),
                showlegend=False
            )
        )

# 设置x轴和y轴
fig.update_xaxes(title_text='时间', tickformat='%H:%M')
fig.update_yaxes(title_text='座位', tickvals=list(range(len(dev_names))), ticktext=dev_names)

# 设置布局
fig.update_layout(
    title='设备预约时间段分布',
    autosize=False,
    width=800,  # 调整图表宽度
    height=2000,  # 调整图表高度
    margin=dict(l=100, r=50, pad=4),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

# 显示图表
fig.show()